In [ ]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import copy

In [ ]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
        
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Convert string column to float
def str_column_to_float(dataset, column):
    
    column = column
    dataset_copy = copy.deepcopy(dataset)
    
    for row in dataset_copy:
        row[column] = float(row[column].strip())
    
    return dataset_copy

In [ ]:
# load and prepare data
filename = '../../data_sets/sonar.all-data.csv'
dataset = load_csv(filename)

# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
    dataset = str_column_to_float(dataset, i)

In [ ]:
def gini_index(grupos, valores_alvo):
    """
    Calcular o indíce-Gini pelo dados diversão
    
    :param grupos: O grupo de registros
    :param valores_alvo: O conjuncto de alvos
    
    :return: gini, Float a pontuação de pureza
    """
    
    gini = 0.0
    for valor_alvo in valores_alvo:
        for grupo in grupos:
            tomanho = len(grupo)
            if tomanho == 0:
                continue
            proporção = [linha[-1] for linha in grupo].count(valores_alvo) / float(tomanho)
            gini += (proporção * (1.0 - proporção))
    return gini
 
def obter_melhor_diversão(dados, n_características=None):
    """
    Obter a melhor diversão pelo dados
    
    :param dados: List, o conjuncto de dados
    :param n_características: Int, o numero de características para usar, quando você está usando a árvore sozinha fica 
    esta entrada em None
    """
    
    valores_alvo = list(set(row[-1] for row in dados)) # Lista único de valores-alvo 
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    
    if n_características is None:
        n_características = len(dados) - 1
    
    # Faz uma lista de características únicos para usar
    features = list()
    while len(features) < n_features:
        indíce = randrange(len(dados[0])-1)
        if indíce not in features:
            features.append(indíce)
    
    
    for indíce in features:
        for registro in dados:
            groups = test_split(indíce, registro[indíce], dados)
            gini = gini_index(groups, valores_alvo)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = indíce, registro[indíce], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}
 
def to_terminal(group):
    """
    Voltar o valor alvo para uma grupo no fim de uma filial
    
    :param grupo: O conjuncto de registros em um lado de uma divisão
    
    :return: valor_de_alvo, Int 
    """
    
    outcomes = [row[-1] for row in group]
    valor_de_alvo = max(set(outcomes), key=outcomes.count)
    return valor_de_alvo
 
def split(nó_atual, max_depth, min_size, n_features, depth):
    """
    Recursivo, faz subdivisões por um nó ou faz um terminal

    :param nó_atual: o nó estar analisado agora, vai mudar o root
    :param 
    """
    
    left, right = nó_atual['groups']
    del(nó_atual['groups'])
    
    # check for a no split
    if not left or not right:
        nó_atual['left'] = nó_atual['right'] = to_terminal(left + right)
        return
    
    # check for max depth
    if depth >= max_depth:
        nó_atual['left'], nó_atual['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size:
        nó_atual['left'] = to_terminal(left)
    else:
        nó_atual['left'] = obter_melhor_diversão(left, n_features)
        split(nó_atual['left'], max_depth, min_size, n_features, depth+1)

    # process right child
    if len(right) <= min_size:
        nó_atual['right'] = to_terminal(right)
    else:
        nó_atual['right'] = obter_melhor_diversão(right, n_features)
        split(nó_atual['right'], max_depth, min_size, n_features, depth+1)

def test_split(indíce, valor, dados):
    """
    Dividir o dados sobre uma característica e o valor da caracaterística dele
    
    :param indíce: Int, o indíce da característica
    :param valor: Float, o valor do indíce por um registro
    :param dados: List, o conjuncto de dados
    
    :return: esquerda, direitaç duas listas de registros dividou de o valor de característica
    """
    
    esquerda, direita = list(), list()
    for linha in dados:
        if linha[indíce] < valor:
            esquerda.append(linha)
        else:
            direita.append(linha)
    return esquerda, direita
        
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = obter_melhor_diversão(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [ ]:
n_features = len(dataset[0])-1
max_depth = 10
min_size = 1

dados_divisão = int(len(dataset)*.8)
trem = dataset[:dados_divisão]
teste = dataset[dados_divisão:]

árvore = build_tree(trem, max_depth, min_size, n_features)

In [ ]:
indíce_teste = 8

print(predict(árvore, teste[indíce_teste]))
print(teste[indíce_teste][-1])

In [ ]:
árvore

In [ ]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [ ]:
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

In [ ]:
def split(nó, max_depth, min_size, n_features, depth):
    
    nó_atual = copy.deepcopy(nó)
    
    left, right = nó_atual['groups']
    del(nó_atual['groups'])
    
    # check for a no split
    if not left or not right:
        nó_atual['left'] = nó_atual['right'] = to_terminal(left + right)
        return
    
    # check for max depth
    if depth >= max_depth:
        nó_atual['left'], nó_atual['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size:
        nó_atual['left'] = to_terminal(left)
    else:
        nó_atual['left'] = obter_melhor_diversão(left, n_features)
        split(nó_atual['left'], max_depth, min_size, n_features, depth+1)

    # process right child
    if len(right) <= min_size:
        nó_atual['right'] = to_terminal(right)
    else:
        nó_atual['right'] = obter_melhor_diversão(right, n_features)
        split(nó_atual['right'], max_depth, min_size, n_features, depth+1)